In [ ]:
!pip install ultralytics wandb roboflow -q

In [ ]:
%mkdir /kaggle/working/datasets
%mkdir /kaggle/working/datasets/coco

In [ ]:
%cd /kaggle/working/datasets/coco

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="Nj0TPvJWofa4sHFzyt5L")
project = rf.workspace("kelvin-snkig").project("ppe-akpdb")
version = project.version(2)
dataset_coco = version.download("coco")

In [ ]:
%mkdir /kaggle/working/datasets/yolo

In [ ]:
%cd /kaggle/working/datasets/yolo

In [ ]:
dataset_yolo = version.download("yolov11")

In [ ]:
%cd /kaggle/working

In [ ]:
from ultralytics import settings

settings.update({"wandb": True})

print(settings['wandb'])

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [ ]:
import wandb

wandb.login(key=user_secrets.get_secret("wandb"))

In [ ]:
project = "PPE Detection Model"

In [ ]:
api = wandb.Api()

runs = api.runs(f'kelpinnnnnn/{project}')

run_ids = []

for run in runs[:10]:
    print(run.id, run.name)
    run_ids.append(run.id)

In [ ]:
for id in run_ids:
    try:
        # Coba download dari project pertama
        artifact = api.artifact(f'kelpinnnnnn/{project}/run_{id}_model:v0', type='model')
        artifact.download()
        print(f"✅ Success: {id} added to run_ids from {project}")
    except Exception as e1:
        print(f"❌ Failed from {project} for run {id}: {e1}")
        try:
            # Coba ulang dari project kedua jika yang pertama gagal
            print("🔁 Trying Ultralytics project...")
            artifact = api.artifact(f'kelpinnnnnn/Ultralytics/run_{id}_model:v0', type='model')
            artifact.download()
            print(f"✅ Success: {id} added to run_ids from Ultralytics")
        except Exception as e2:
            print(f"❌ Failed from Ultralytics for run {id}: {e2}")

In [ ]:
import json

# Load annotation file
with open(f"{dataset_coco.location}/valid/_annotations.coco.json") as f:
    annotation_data = json.load(f)

# Buat mapping dari file_name ke image_id
file_to_id = {
    img["file_name"].split('.')[0]: img["id"]
    for img in annotation_data["images"]
}

In [ ]:
from ultralytics import YOLO  # Ultralytics YOLO
from pathlib import Path
import os
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json

for id in run_ids:
    run = api.run(f"kelpinnnnnn/{project}/{id}")
    wandb.init(project="YOLO-PPE Model Testing", name=run.name)

    model_path=f"/kaggle/working/artifacts/run_{id}_model:v0"
    # Load the YOLO model
    model = YOLO(f"{model_path}/best.pt")

    # Run evaluation (Automatically computes confusion matrix)
    results = model.val(data=f"{dataset_yolo.location}/data.yaml", project=f"{run.name}", name="results", save_json=True)

    # Parent directory where all results folders are saved
    results_base_dir = Path(f"/kaggle/working/{run.name}")

    # Find all subdirectories that start with "results"
    results_dirs = [d for d in results_base_dir.iterdir() if d.is_dir() and d.name.startswith("results")]

    # Sort them by modification time (most recent last)
    results_dirs.sort(key=os.path.getmtime)

    # Get the latest results directory
    save_dir = results_dirs[-1] if results_dirs else None
 
    image_files = [
        "F1_curve.png", "PR_curve.png", "P_curve.png", "R_curve.png", "confusion_matrix.png",
        "val_batch0_labels.jpg", "val_batch0_pred.jpg",
        "val_batch1_labels.jpg", "val_batch1_pred.jpg",
        "val_batch2_labels.jpg", "val_batch2_pred.jpg"
    ]

    # Log images only if they exist
    images = {file.split(".")[0]: wandb.Image(str(save_dir / file)) for file in image_files if (save_dir / file).exists()}

    with open(save_dir / "predictions.json") as f:
        predictions = json.load(f)

    # Ganti string image_id dengan int dari mapping
    for pred in predictions:
        image_name = pred["image_id"]  # misalnya 'ppe_0909_jpg.rf.107c3a1c8eabd944242340516b93d92c'
        image_key = image_name.split('.')[0]  # hapus ekstensi

        if image_key in file_to_id:
            pred["image_id"] = file_to_id[image_key]
        else:
            print(f"❌ Image {image_key} not found in annotations")

    # Simpan ulang file prediction yang sudah diperbaiki
    with open(save_dir / "fixed_prediction.json", "w") as f:
        json.dump(predictions, f)

    coco_gt = COCO(f"{dataset_coco.location}/valid/_annotations.coco.json")
    coco_dt = coco_gt.loadRes(f"{save_dir}/fixed_prediction.json")

    coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    metrics = {
        "speed/inference": results.speed['inference'],
        "speed/loss": results.speed['loss'],
        "speed/preprocess": results.speed['preprocess'],
        "speed/postprocess": results.speed['postprocess'],
        "metrics/mAP50(B)": results.box.map50,
        "metrics/mAP50-95(B)": results.box.map,
        "metrics/precision(B)": results.box.mp, 
        "metrics/recall(B)": results.box.mr,
        "metrics/fitness": results.fitness,
        "metrics/AP50 (Small)": coco_eval.stats[3],
        "metrics/AP50 (Medium)": coco_eval.stats[4],
        "metrics/AP50 (Large)": coco_eval.stats[5]
    }
    # Log all metrics and images in a single call
    wandb.log({**metrics, **images})
    
    wandb.finish()